In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import zip_longest
import os
import glob

In [2]:
# data = json.load(open("2025-06-15_11-38-56-781937_measure_weight_FORCE_MEASUREMENT_CASE1.json", encoding="utf-8"))
data = json.load(open("2025-06-15_11-39-04-387039_measure_weight_FORCE_MEASUREMENT_CASE.json", encoding="utf-8"))

df = pd.json_normalize(
    data,
    record_path="raw_measurements",
    meta=["version","type","start_time","end_time","sampling_rate","weight_filter","zero_offset","expected_weight","robot_type","force_sensor_offset"],
    errors="ignore"
)
df['timestamp'] = df['timestamp'] - df['timestamp'][0]
print(df.shape)
print(df.keys())
print(df.head())

(493, 19)
Index(['timestamp', 'value', 'force_vector', 'robot_tcp.timestamp',
       'robot_tcp.flange', 'robot_tcp.tcp_offset', 'robot_tcp.velocity_linear',
       'robot_tcp.velocity_angular', 'robot_tcp.joint_angles', 'version',
       'type', 'start_time', 'end_time', 'sampling_rate', 'weight_filter',
       'zero_offset', 'expected_weight', 'robot_type', 'force_sensor_offset'],
      dtype='object')
   timestamp  value          force_vector  robot_tcp.timestamp  \
0   0.000000 -0.054   [0.603, 0.618, 9.0]         1.749980e+09   
1   0.001043 -0.055    [0.647, 0.59, 9.0]         1.749980e+09   
2   0.002042 -0.055   [0.683, 0.56, 10.0]         1.749980e+09   
3   0.003042 -0.055  [0.711, 0.529, 11.0]         1.749980e+09   
4   0.004042 -0.055   [0.73, 0.502, 11.0]         1.749980e+09   

                                    robot_tcp.flange  \
0  [[-0.5583840012550354, -0.3735954165458679, -0...   
1  [[-0.5583451986312866, -0.37359368801116943, -...   
2  [[-0.5583451986312866, -

In [3]:
# input_folder = "All measurements V2"
# output_folder = "All measurements V2, cleaned"

# # make sure output folder exists
# os.makedirs(output_folder, exist_ok=True)

# files = glob.glob(os.path.join(input_folder, "*.json"))

# parts = []  # collect accepted dataframes
# run_counter = 0

# for file in files:
#     print(f"\nProcessing: {file}")
    
#     try:
#         # load JSON
#         with open(file, encoding="utf-8") as f:
#             data = json.load(f)

#         # flatten JSON into dataframe
#         df = pd.json_normalize(
#             data,
#             record_path="raw_measurements",
#             meta=["version","type","start_time","end_time","sampling_rate","weight_filter","zero_offset","expected_weight","robot_type","force_sensor_offset"],
#             errors="ignore"
#         )

#         df['timestamp'] = df['timestamp'] - df['timestamp'].iloc[0]

#         # checks
#         if len(df) < 400:
#             print(f"⚠️ Skipping {file} — only {len(df)} rows")
#             continue
            
#         if "value" not in df.columns:
#             print(f"⚠️ Skipping {file} — no 'value' column")
#             continue

#         if df['value'].isna().all() or (df['value'] == 0).all():
#             print(f"⚠️ Skipping {file} — all values are 0 or NaN")
#             continue

#         # assign sequential run index
#         run_counter += 1
#         df["run_index"] = run_counter

#         parts.append(df)
#         print(f"✅ Accepted {file} ({len(df)} rows) as run_index={run_counter}")

#     except Exception as e:
#         print(f"⚠️ Skipping file {file}, error: {e}")
#         continue

# # write ONE CSV in the current folder
# if not parts:
#     print("No valid measurements found. Nothing written.")
# else:
#     all_long = pd.concat(parts, ignore_index=True)
#     all_long = all_long.sort_values(["run_index", "timestamp"])
#     final_csv = "all_measurements.csv"
#     all_long.to_csv(final_csv, index=False, encoding="utf-8")
#     print(f"✅ Wrote {len(all_long):,} rows to {os.path.abspath(final_csv)}")

In [4]:
input_folder = "All measurements V2"
output_folder = "All measurements V2, cleaned"
BATCH_SIZE = 5000  # files (measurements) per flush

os.makedirs(output_folder, exist_ok=True)  # ensure folder exists
final_csv = os.path.join(output_folder, "all_measurements_correct.csv")  # <-- final file name

files = glob.glob(os.path.join(input_folder, "*.json"))

run_counter = 0
wrote_header = False
batch = []
columns_order = None  # will lock on first batch write
accepted_in_batch = 0

for file in files:
    try:
        with open(file, encoding="utf-8") as f:
            data = json.load(f)

        df = pd.json_normalize(
            data,
            record_path="raw_measurements",
            meta=["version","type","start_time","end_time","sampling_rate",
                  "weight_filter","zero_offset","expected_weight","robot_type","force_sensor_offset"],
            errors="ignore"
        )

        df["timestamp"] = df["timestamp"] - df["timestamp"].iloc[0]

        # your checks
        if len(df) < 400:               continue
        if "value" not in df.columns:   continue
        if df["value"].isna().all() or (df["value"] == 0).all():  continue

        # ids
        df["measurement_id"] = os.path.splitext(os.path.basename(file))[0]
        run_counter += 1
        df["run_index"] = run_counter

        batch.append(df)
        accepted_in_batch += 1

        # flush every BATCH_SIZE accepted files
        if accepted_in_batch >= BATCH_SIZE:
            out = pd.concat(batch, ignore_index=True)

            # lock column order on first write; enforce consistently thereafter
            if columns_order is None:
                columns_order = out.columns.tolist()
            else:
                out = out.reindex(columns=columns_order, fill_value=pd.NA)

            out.to_csv(final_csv, mode="a", index=False, encoding="utf-8",
                       header=not wrote_header)
            wrote_header = True

            # clear memory
            batch.clear()
            accepted_in_batch = 0
            del out
            gc.collect()

    except Exception:
        continue

# flush remainder
if batch:
    out = pd.concat(batch, ignore_index=True)
    if columns_order is None:
        columns_order = out.columns.tolist()
    else:
        out = out.reindex(columns=columns_order, fill_value=pd.NA)
    out.to_csv(final_csv, mode="a", index=False, encoding="utf-8",
               header=not wrote_header)

print(f"Done. Appended to {os.path.abspath(final_csv)}")

Done. Appended to C:\Users\piete\Documents\Smart Robotics PROJECT\All measurements V2, cleaned\all_measurements_correct.csv


In [5]:
df = pd.read_csv("All measurements V2, cleaned/all_measurements_correct.csv")

In [6]:
df

,value,force_vector,timestamp,robot_tcp.tcp_offset,robot_tcp.flange,robot_tcp.velocity_linear,robot_tcp.velocity_angular,robot_tcp.timestamp,robot_tcp.joint_angles,version,...,start_time,end_time,sampling_rate,weight_filter,zero_offset,expected_weight,robot_type,force_sensor_offset,measurement_id,run_index
0,0.324,"[0.775, -0.453, 20.0]",0.000000,"[[0.0, 0.0, 0.4], [1.0, 0.0, 0.0, 0.0]]","[[-0.37170541286468506, -0.5985541343688965, 0...","[0.07976639158614517, -0.010369553826843979, 0...","[2.747849505768034e-05, -0.005977424159985412,...",1.758967e+09,"[0.7639275193214417, -0.9689134520343323, -4.1...",2,...,1.758967e+09,1.758967e+09,1000.0,NaN,-0.016568,0.335,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-09-27_11-53-22-904561_measure_weight_FORC...,1
1,0.324,"[0.785, -0.431, 20.0]",0.001000,"[[0.0, 0.0, 0.4], [1.0, 0.0, 0.0, 0.0]]","[[-0.37170541286468506, -0.5985541343688965, 0...","[0.07976639158614517, -0.010369553826843979, 0...","[2.747849505768034e-05, -0.005977424159985412,...",1.758967e+09,"[0.7639275193214417, -0.9689134520343323, -4.1...",2,...,1.758967e+09,1.758967e+09,1000.0,NaN,-0.016568,0.335,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-09-27_11-53-22-904561_measure_weight_FORC...,1
2,0.325,"[0.8, -0.402, 20.0]",0.002000,"[[0.0, 0.0, 0.4], [1.0, 0.0, 0.0, 0.0]]","[[-0.3713288903236389, -0.5986536741256714, 0....","[0.1536763275358725, -0.04416399779851208, 0.8...","[-0.016156653637422146, 0.011442821817490728, ...",1.758967e+09,"[0.768875777721405, -1.0236349266818543, -4.13...",2,...,1.758967e+09,1.758967e+09,1000.0,NaN,-0.016568,0.335,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-09-27_11-53-22-904561_measure_weight_FORC...,1
3,0.325,"[0.81, -0.368, 20.0]",0.003000,"[[0.0, 0.0, 0.4], [1.0, 0.0, 0.0, 0.0]]","[[-0.3713288903236389, -0.5986536741256714, 0....","[0.1536763275358725, -0.04416399779851208, 0.8...","[-0.016156653637422146, 0.011442821817490728, ...",1.758967e+09,"[0.768875777721405, -1.0236349266818543, -4.13...",2,...,1.758967e+09,1.758967e+09,1000.0,NaN,-0.016568,0.335,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-09-27_11-53-22-904561_measure_weight_FORC...,1
4,0.326,"[0.815, -0.338, 19.0]",0.003999,"[[0.0, 0.0, 0.4], [1.0, 0.0, 0.0, 0.0]]","[[-0.37108150124549866, -0.5987440943717957, 0...","[0.156813803573975, -0.048874791685071466, 0.8...","[-0.02211244230129169, 0.016721851482539372, -...",1.758967e+09,"[0.7692531943321228, -1.026842550640442, -4.13...",2,...,1.758967e+09,1.758967e+09,1000.0,NaN,-0.016568,0.335,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-09-27_11-53-22-904561_measure_weight_FORC...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15179139,0.399,"[0.19, 0.668, 16126.0]",0.495248,"[[0.0, 0.0, 0.4], [1.0, 0.0, 0.0, 0.0]]","[[-0.5305851101875305, -0.18714310228824615, 0...","[0.22661341902108, 0.41108814955365547, 0.6861...","[-0.018644322339820916, -0.041922610040263236,...",1.759916e+09,"[3.795844793319702, -1.3299107414535065, -2.00...",2,...,1.759916e+09,1.759916e+09,1000.0,NaN,-0.168018,0.482,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-10-08_11-30-00-234675_measure_weight_FORC...,30973
15179140,0.400,"[0.23, 0.691, 16322.0]",0.496248,"[[0.0, 0.0, 0.4], [1.0, 0.0, 0.0, 0.0]]","[[-0.5305851101875305, -0.18714310228824615, 0...","[0.22661341902108, 0.41108814955365547, 0.6861...","[-0.018644322339820916, -0.041922610040263236,...",1.759916e+09,"[3.795844793319702, -1.3299107414535065, -2.00...",2,...,1.759916e+09,1.759916e+09,1000.0,NaN,-0.168018,0.482,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-10-08_11-30-00-234675_measure_weight_FORC...,30973
15179141,0.402,"[0.26, 0.711, 16660.0]",0.497248,"[[0.0, 0.0, 0.4], [1.0, 0.0, 0.0, 0.0]]","[[-0.5301331877708435, -0.18638724088668823, 0...","[0.22661341902108, 0.41108814955365547, 0.6861...","[-0.018644322339820916, -0.041922610040263236,...",1.759916e+09,"[3.795844793319702, -1.3299107414535065, -2.00...",2,...,1.759916e+09,1.75

In [7]:
df.keys()

Index(['value', 'force_vector', 'timestamp', 'robot_tcp.tcp_offset',
       'robot_tcp.flange', 'robot_tcp.velocity_linear',
       'robot_tcp.velocity_angular', 'robot_tcp.timestamp',
       'robot_tcp.joint_angles', 'version', 'type', 'start_time', 'end_time',
       'sampling_rate', 'weight_filter', 'zero_offset', 'expected_weight',
       'robot_type', 'force_sensor_offset', 'measurement_id', 'run_index'],
      dtype='object')